In [1]:
import numpy as np
import decimal

In [2]:
def CM(y_test,y_test_obs):
        '''
        Prints confusion matrix 
        y_test is list of y values in the test dataset
        y_test_obs is list of y values predicted by the model

        '''

        for i in range(len(y_test_obs)):
            if(y_test_obs[i]>0.6):
                y_test_obs[i]=1
            else:
                y_test_obs[i]=0

        cm=[[0,0],[0,0]]
        fp=0
        fn=0
        tp=0
        tn=0

        for i in range(len(y_test)):
            if(y_test[i]==1 and y_test_obs[i]==1):
                tp=tp+1
            if(y_test[i]==0 and y_test_obs[i]==0):
                tn=tn+1
            if(y_test[i]==1 and y_test_obs[i]==0):
                fp=fp+1
            if(y_test[i]==0 and y_test_obs[i]==1):
                fn=fn+1
        cm[0][0]=tn
        cm[0][1]=fp
        cm[1][0]=fn
        cm[1][1]=tp
        acc=(tp+tn)/(tp+tn+fn+fp)
        p= tp/(tp+fp)
        r=tp/(tp+fn)
        f1=(2*p*r)/(p+r)

        print("Confusion Matrix : ")
        print(cm)
        print("\n")
        print(f"Precision : {p}")
        print(f"Accuracy : {acc}")
        print(f"Recall : {r}")
        print(f"F1 SCORE : {f1}")
        return acc

In [3]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

def tanh(x):
    return np.tanh(x)

def tanh_prime(x):
    return 1-np.tanh(x)**2


def sigmoid(x):
    return 1/(1+np.exp(-x))

def sigmoid_prime(x):
    return sigmoid(x)*(1-sigmoid(x))

def relu(x):
    return np.maximum(0,x)
def relu_prime(x):
    x[x<=0] = 0
    x[x>0] = 1
    return x
def softmax(x):
    e = np.exp(x)
    return e / np.sum(e, axis=1)
def softmax_prime(x):
    
    return jacobian_m

In [4]:
def mse(y_true, y_pred):
    return np.mean(np.power(y_true-y_pred, 2))

def mse_prime(y_true, y_pred):
    return 2*(y_pred-y_true)/y_true.size

In [5]:
class Layer:
    def __init__(self):
        self.input = None
        self.output = None

    # computes the output Y of a layer for a given input X
    def forward_propagation(self, input):
        raise NotImplementedError

    # computes dE/dX for a given dE/dY (and update parameters if any)
    def backward_propagation(self, output_error, learning_rate):
        raise NotImplementedError

In [6]:
class ActivationLayer(Layer):
    def __init__(self, activation, activation_prime):
        self.activation = activation
        self.activation_prime = activation_prime

    # returns the activated input
    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = self.activation(self.input)
        return self.output

    # Returns input_error=dE/dX for a given output_error=dE/dY.
    # learning_rate is not used because there is no "learnable" parameters.
    def backward_propagation(self, output_error, learning_rate):
        return self.activation_prime(self.input) * output_error

In [7]:
class FCLayer(Layer):
    # input_size = number of input neurons
    # output_size = number of output neurons
    def __init__(self, input_size, output_size):
        self.weights = np.random.rand(input_size, output_size) - 0.5
        self.bias = np.random.rand(1, output_size) - 0.5

    # returns output for a given input
    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = np.dot(self.input, self.weights) + self.bias
        return self.output

    # computes dE/dW, dE/dB for a given output_error=dE/dY. Returns input_error=dE/dX.
    def backward_propagation(self, output_error, learning_rate):
        input_error = np.dot(output_error, self.weights.T)
        weights_error = np.dot(self.input.T, output_error)
        # dBias = output_error

        # update parameters
        self.weights -= learning_rate * weights_error
        self.bias -= learning_rate * output_error
        return input_error

In [52]:
class Network:
    def __init__(self):
        self.layers = []
        self.loss = None
        self.loss_prime = None

    # add layer to network
    def add(self, layer):
        self.layers.append(layer)

    # set loss to use
    def use(self, loss, loss_prime):
        self.loss = loss
        self.loss_prime = loss_prime

    # predict output for given input
    def predict(self, input_data):
        # sample dimension first
        samples = len(input_data)
        result = []

        # run network over all samples
        for i in range(samples):
            # forward propagation
            output = input_data[i]
            for layer in self.layers:
                output = layer.forward_propagation(output)
            result.append(output)

        return result

    # train the network
    def fit(self, x_train, y_train, epochs, learning_rate):
        # sample dimension first
        samples = len(x_train)

        # training loop
        for i in range(epochs):
            err = 0
            for j in range(samples):
                # forward propagation
                output = x_train[j]
                for layer in self.layers:
                    output = layer.forward_propagation(output)

                # compute loss (for display purpose only)
                err += self.loss(y_train[j], output)

                # backward propagation
                error = self.loss_prime(y_train[j], output)
                for layer in reversed(self.layers):
                    error = layer.backward_propagation(error, learning_rate)

            # calculate average error on all samples
            err /= samples
            print('epoch %d/%d   error=%f' % (i+1, epochs, err))

In [53]:
FCLayer(7,14).bias

array([[-0.39862982, -0.05979862,  0.01871812,  0.21240248,  0.47083212,
        -0.39194168, -0.33689575, -0.1067865 ,  0.04990504,  0.07590178,
        -0.31588287, -0.30024662, -0.24377002, -0.33167475]])

In [54]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

df = pd.read_csv('cleaned1.csv')
df=(df-df.min())/(df.max()-df.min())
#del df['Delivery phase']
#del df['Education']
#df = df.drop(['Education'],axis=1)
#df = df.drop(['Delivery phase'],axis=1)

X = df.loc[:,'Community':'Residence'].values
Y = df.loc[:,'Result'].values

#X=df[df.columns[:-1]]
#Y=df[df.columns[-1]]
    
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state = 42)

In [55]:
x_train=x_train.reshape(len(x_train),1,7)
y_train=y_train.reshape(len(y_train),1,1)

In [56]:
zeros=[]
for i in list(y_train):
    if i==0:
        zeros.append(i)
print(len(zeros))
print(len(y_train))

66
117


In [59]:
net = Network()
net.add(FCLayer(7, 12))
net.add(ActivationLayer(relu, relu_prime))
net.add(FCLayer(12,24))
net.add(ActivationLayer(relu, relu_prime))
net.add(FCLayer(24,8))
net.add(ActivationLayer(relu , relu_prime))
net.add(FCLayer(8,1))
net.add(ActivationLayer(sigmoid, sigmoid_prime))
net.use(mse, mse_prime)
net.fit(x_train, y_train, epochs=100, learning_rate=0.05)
out = np.asarray(net.predict(x_test))
accu=CM(y_test,out)
print(accu)

epoch 1/100   error=0.249795
epoch 2/100   error=0.239902
epoch 3/100   error=0.226834
epoch 4/100   error=0.201138
epoch 5/100   error=0.162103
epoch 6/100   error=0.133031
epoch 7/100   error=0.118056
epoch 8/100   error=0.113574
epoch 9/100   error=0.111554
epoch 10/100   error=0.109564
epoch 11/100   error=0.109044
epoch 12/100   error=0.108724
epoch 13/100   error=0.108507
epoch 14/100   error=0.108250
epoch 15/100   error=0.108075
epoch 16/100   error=0.107951
epoch 17/100   error=0.108037
epoch 18/100   error=0.107877
epoch 19/100   error=0.107681
epoch 20/100   error=0.107759
epoch 21/100   error=0.107636
epoch 22/100   error=0.107459
epoch 23/100   error=0.107487
epoch 24/100   error=0.107475
epoch 25/100   error=0.107373
epoch 26/100   error=0.107301
epoch 27/100   error=0.107155
epoch 28/100   error=0.107196
epoch 29/100   error=0.107198
epoch 30/100   error=0.107112
epoch 31/100   error=0.107056
epoch 32/100   error=0.107001
epoch 33/100   error=0.106892
epoch 34/100   erro

In [51]:
out

array([[[0.]],

       [[1.]],

       [[0.]],

       [[1.]],

       [[0.]],

       [[0.]],

       [[0.]],

       [[1.]],

       [[0.]],

       [[1.]],

       [[1.]],

       [[0.]],

       [[0.]],

       [[1.]],

       [[1.]],

       [[1.]],

       [[1.]],

       [[0.]],

       [[1.]],

       [[0.]],

       [[1.]],

       [[1.]],

       [[1.]],

       [[1.]],

       [[1.]],

       [[1.]],

       [[1.]],

       [[0.]],

       [[1.]],

       [[1.]],

       [[1.]],

       [[1.]],

       [[0.]],

       [[0.]],

       [[1.]],

       [[0.]],

       [[1.]],

       [[1.]],

       [[0.]],

       [[0.]],

       [[1.]],

       [[0.]],

       [[1.]],

       [[0.]],

       [[0.]],

       [[0.]],

       [[1.]],

       [[1.]],

       [[0.]],

       [[1.]],

       [[0.]]])

In [121]:
max(y_matplot)

ValueError: max() arg is an empty sequence

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(x_matplot,y_matplot)
plt.show()

In [ ]:
x_matplot=[]
y_matplot=[]
from numpy import arange